In [50]:
!pip install nltk gensim


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [51]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# EDA

In [52]:
df = pd.read_csv("../data/gpt_fake_recommendations.csv")

In [53]:
df.head()

,Company,Short Name,Date,Target Price,Recommendation,Commentary
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth in cost management, c..."
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL is benefiting from investment in research...
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,AAPL's performance in technology has been stea...
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties in shift in consumer ...
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges in expansion into internatio...


In [54]:
df.value_counts("Recommendation")

Recommendation
Buy     434
Sell    428
Hold    374
Name: count, dtype: int64

In [55]:
df.value_counts("Company")

Company
3M Company                           12
Norfolk Southern Corporation         12
Salesforce, Inc.                     12
S&P Global Inc.                      12
Raytheon Technologies Corporation    12
                                     ..
Exelon Corporation                   12
Eli Lilly and Company                12
Dow Inc.                             12
Deere & Company                      12
Zoetis Inc.                          12
Name: count, Length: 103, dtype: int64

In [56]:
df.groupby(["Short Name"]).mean("Target Price")

,Target Price
Short Name,
AAPL,249.782500
ABBV,227.696667
ADBE,251.938333
ALL,280.011667
AMGN,294.452500
...,...
VZ,259.306667
WFC,246.332500
WMT,251.254167


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company         1236 non-null   object 
 1   Short Name      1236 non-null   object 
 2   Date            1236 non-null   object 
 3   Target Price    1236 non-null   float64
 4   Recommendation  1236 non-null   object 
 5   Commentary      1236 non-null   object 
dtypes: float64(1), object(5)
memory usage: 58.1+ KB


# Removing Stopwords

In [58]:
df["Commentary"] = df["Commentary"].apply(strip_multiple_whitespaces)

In [59]:
# Show list of stopwords
nltk_stopwords = set(stopwords.words('english'))
print(sorted(nltk_stopwords))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some',

In [60]:
# Remove negations from the list
to_remove=['ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'don', "don't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [61]:
for word in to_remove:
    nltk_stopwords.remove(word)

In [62]:
print(sorted(nltk_stopwords))

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'd', 'did', 'do', 'does', 'doing', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'she', "she's", 'should', "should've", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'who

In [63]:
# Correct apply version
df["Commentary"] = df["Commentary"].apply(lambda comment: remove_stopwords(comment, stopwords=nltk_stopwords))

In [64]:
print(df)

                 Company Short Name        Date  Target Price Recommendation  \
0             Apple Inc.       AAPL  2023-01-15        121.35           Hold   
1             Apple Inc.       AAPL  2023-02-15        371.59            Buy   
2             Apple Inc.       AAPL  2023-03-15        202.52           Hold   
3             Apple Inc.       AAPL  2023-04-15        355.88           Hold   
4             Apple Inc.       AAPL  2023-05-15        292.45           Sell   
...                  ...        ...         ...           ...            ...   
1231  Exelon Corporation        EXC  2023-08-15        242.37           Sell   
1232  Exelon Corporation        EXC  2023-09-15        248.48            Buy   
1233  Exelon Corporation        EXC  2023-10-15        148.36           Hold   
1234  Exelon Corporation        EXC  2023-11-15        143.41            Buy   
1235  Exelon Corporation        EXC  2023-12-15        172.69            Buy   

                                       

# Add dates for 3 / 6 / 9 / 12 months

In [65]:
# transform data column to actual date
df["Date"] = pd.to_datetime(df["Date"],format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Company         1236 non-null   object        
 1   Short Name      1236 non-null   object        
 2   Date            1236 non-null   datetime64[ns]
 3   Target Price    1236 non-null   float64       
 4   Recommendation  1236 non-null   object        
 5   Commentary      1236 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 58.1+ KB


In [88]:
# Apply DateOffset for each row using 'apply' on the DataFrame
df["Date after 3 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=3))
df["Date after 6 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=6))
df["Date after 9 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=9))
df["Date after 12 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=12))


In [90]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-04-15,2023-07-15,2023-10-15,2024-01-15
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-05-15,2023-08-15,2023-11-15,2024-02-15
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-06-15,2023-09-15,2023-12-15,2024-03-15
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-07-15,2023-10-15,2024-01-15,2024-04-15
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-08-15,2023-11-15,2024-02-15,2024-05-15
...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-11-15,2024-02-15,2024-05-15,2024-08-15
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-12-15,2024-03-15,2024-06-15,2024-09-15
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2024-01-15,2024-04-15,2024-07-15,2024-10-15
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2024-02-15,2024-05-15,2024-08-15,2024-11-15


# API Connection

In [89]:
import yfinance as yf
from datetime import timedelta
import pandas as pd
import numpy as np  # Import numpy to use np.abs

def get_stock_price_nearest(ticker, date):
    try:
        # Ensure the date is in Pandas Timestamp format
        target_date = pd.to_datetime(date)

        # Define a range around the target date to fetch data
        start_date = target_date - timedelta(days=5)
        end_date = target_date + timedelta(days=5)

        # Download data within this range
        stock_data = yf.download(ticker, start=start_date, end=end_date)

        if not stock_data.empty:
            # Check if data exists for the exact date
            if target_date in stock_data.index:
                return stock_data.loc[target_date]['Close']
            else:
                # Manually find the closest available trading day using numpy.abs()
                stock_data['diff'] = np.abs(stock_data.index - target_date)
                closest_date = stock_data['diff'].idxmin()  # Get the index of the closest date
                closest_price = stock_data.loc[closest_date]['Close']
                return closest_price
        else:
            return f"No data found for {ticker} within the date range {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}."
    
    except Exception as e:
        return f"An unexpected error occurred for {ticker}: {str(e)}"


In [92]:
short_name = 'TSLA'
date = '2024-12-15'

price = get_stock_price_nearest(short_name, date)
print(price)


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-12-10 00:00:00 -> 2024-12-20 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1733806800, endDate = 1734670800")')


No data found for TSLA within the date range 2024-12-10 to 2024-12-20.


In [87]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2024-03-15,2024-06-15,2024-09-15,2024-12-15
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2024-03-15,2024-06-15,2024-09-15,2024-12-15
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2024-03-15,2024-06-15,2024-09-15,2024-12-15
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2024-03-15,2024-06-15,2024-09-15,2024-12-15
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2024-03-15,2024-06-15,2024-09-15,2024-12-15
...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2024-03-15,2024-06-15,2024-09-15,2024-12-15
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2024-03-15,2024-06-15,2024-09-15,2024-12-15
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2024-03-15,2024-06-15,2024-09-15,2024-12-15
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2024-03-15,2024-06-15,2024-09-15,2024-12-15


In [93]:
def calculate_prices(row):
    short_name = row['Short Name']
    row['Start Price'] = get_stock_price_nearest(short_name, row['Date'])
    row['Price after 3 Months'] = get_stock_price_nearest(short_name, row['Date after 3 Months'])
    row['Price after 6 Months'] = get_stock_price_nearest(short_name, row['Date after 6 Months'])
    row['Price after 9 Months'] = get_stock_price_nearest(short_name, row['Date after 9 Months'])
    #row['Price after 12 Months'] = get_stock_price_nearest(short_name, row['Date after 12 Months'])
    return row

# Apply the function to each row
df = df.apply(calculate_prices, axis=1)

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

In [94]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months,Start Price,Price after 3 Months,Price after 6 Months,Price after 9 Months
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-04-15,2023-07-15,2023-10-15,2024-01-15,134.759995,165.210007,190.690002,178.720001
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-05-15,2023-08-15,2023-11-15,2024-02-15,155.330002,172.070007,177.449997,188.009995
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-06-15,2023-09-15,2023-12-15,2024-03-15,152.990005,186.009995,175.009995,197.570007
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-07-15,2023-10-15,2024-01-15,2024-04-15,165.210007,190.690002,178.720001,183.630005
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-08-15,2023-11-15,2024-02-15,2024-05-15,172.070007,177.449997,188.009995,183.860001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-11-15,2024-02-15,2024-05-15,2024-08-15,39.59,39.669998,34.860001,38.73
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-12-15,2024-03-15,2024-06-15,2024-09-15,41.650002,35.490002,36.48,35.540001
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2024-01-15,2024-04-15,2024-07-15,2024-10-15,40.169998,35.700001,36.27,35.349998
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2024-02-15,2024-05-15,2024-08-15,2024-11-15,39.669998,34.860001,38.73,37.5


# Topic Modeling with LDA

# FINBERT